This is pulled from https://github.com/snap-stanford/ogb/tree/master/examples/lsc/pcqm4m

dependencies:
- install rdkit (I used conda) https://www.rdkit.org/docs/Install.html
1. conda create -c conda-forge -n my-rdkit-env rdkit
2. conda activate my-rdkit-env
3. cd [anaconda folder]/bin
4. source activate my-rdkit-env

- install ogb, torch and pytorch-geometric

- run the main-gnn.py code to download the dataset, extract and train (see the readme.md). I could finish extracting because my RAM wasn't enough. If you face the some problem, use this notebook to extract just a handful of the dataset. This code is taken from 

In [19]:
from ogb.lsc import DglPCQM4MDataset, PCQM4MEvaluator

import argparse
import dgl
import numpy as np
import os
import random
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm

import pandas as pd

from gnn import GNN

import os
import os.path as osp
import shutil
from ogb.utils import smiles2graph
from ogb.utils.torch_util import replace_numpy_with_torchtensor
from ogb.utils.url import decide_download, download_url, extract_zip
import pandas as pd
import numpy as np
from dgl.data.utils import load_graphs, save_graphs, Subset
import dgl
from tqdm import tqdm
import torch

reg_criterion = torch.nn.L1Loss()

In [2]:
# !mkdir log
# !mkdir checkpoint
# !mkdir test

A subdirectory or file log already exists.
A subdirectory or file checkpoint already exists.
A subdirectory or file test already exists.


In [3]:
# !python main_gnn.py --gnn gcn --log_dir log --checkpoint_dir checkpoint --save_test_dir test

In [5]:
"""
my folder structure looks like (obtained from running main_gnn)
dataset/
    pcqm4m_kddcup2021/
        mapping/
        processed/
        raw/
            data.csv.gz
"""

ROOT = "dataset/pcqm4m_kddcup2021"
filename = "{}/{}".format(ROOT, "raw/data.csv.gz")
data_df = pd.read_csv(filename)
data_df

,idx,smiles,homolumogap
0,0,Cc1ccc(cc1)C1C=c2cnccc2=NC1=O,3.047675
1,1,COc1cc(OC)ccc1C=CN(C(=O)C)C,4.410966
2,2,C=CCN(C(=O)C)C=Cc1ccccc1C,4.639541
3,3,C=CCN(C(=O)C)C=Cc1ccccc1F,4.492600
4,4,C=CCN(C(=O)C)C=Cc1ccccc1Cl,4.612330
...,...,...,...
3803448,3803448,O=N(=O)c1ccc(c(c1)N(=O)=O)Cl,NaN
3803449,3803449,NCC(=O)COP(=O)(O)O,NaN
3803450,3803450,CC(CN)O,NaN
3803451,3803451,OC1C=CC=C(C1O)C(=O)O,NaN


In [9]:
"""
Load Train/Test/Valid split dictionary
"""
# INPUT PARAMETERS HERE: SET TRAIN, TEST AND VAL SIZE
trainSize = 50000
testSize = 10000
valSize = 10000

# load raw split dict
split_dict = torch.load(osp.join(ROOT, 'split_dict.pt'))
# load new split dict
new_split_dict = split_dict.copy()

# sample new split dict
new_split_dict["train"] = np.random.choice(new_split_dict["train"], size=trainSize, replace=False)
new_split_dict["test"] = np.random.choice(new_split_dict["test"], size=testSize, replace=False)
new_split_dict["valid"] = np.random.choice(new_split_dict["valid"], size=valSize, replace=False)

In [10]:
# extract train, test and valid indices
sampleIndices = np.append(
    new_split_dict["train"], [new_split_dict["test"], new_split_dict["valid"]]
)

# remap indices
idMapping = {}
for i, index in enumerate(sampleIndices):
    idMapping[index] = i
    
idMapping

{1692148: 0,
 1206752: 1,
 1289037: 2,
 439210: 3,
 1674787: 4,
 2376971: 5,
 2588628: 6,
 2863056: 7,
 1400392: 8,
 1058168: 9,
 884279: 10,
 1519775: 11,
 2817473: 12,
 1137286: 13,
 1931323: 14,
 2225859: 15,
 1196936: 16,
 184880: 17,
 2073180: 18,
 749131: 19,
 2307840: 20,
 1987005: 21,
 1053980: 22,
 625440: 23,
 2446910: 24,
 56509: 25,
 1118651: 26,
 366434: 27,
 239551: 28,
 912575: 29,
 2884506: 30,
 1394501: 31,
 548592: 32,
 1447302: 33,
 2539445: 34,
 2892602: 35,
 723899: 36,
 2194990: 37,
 459606: 38,
 457292: 39,
 2620823: 40,
 1921112: 41,
 2680664: 42,
 1939067: 43,
 94599: 44,
 912835: 45,
 492857: 46,
 1787638: 47,
 1314356: 48,
 2395150: 49,
 953469: 50,
 1821495: 51,
 260797: 52,
 2312809: 53,
 841230: 54,
 112597: 55,
 1559783: 56,
 332243: 57,
 2216035: 58,
 2409948: 59,
 2331202: 60,
 1174133: 61,
 1269379: 62,
 2345695: 63,
 1355369: 64,
 214856: 65,
 1302997: 66,
 2042714: 67,
 2260688: 68,
 492488: 69,
 2680973: 70,
 1857727: 71,
 330634: 72,
 1940629: 73,


In [11]:
"""
Save New Dataframe based on smaller samples
"""
# filter dataframe
sampleDF = data_df.iloc[sampleIndices]
sampleDF["old_idx"] = sampleDF["idx"]
# remap the index
sampleDF["idx"] = sampleDF["idx"].apply(lambda x: idMapping[x])
sampleDF.index = sampleDF["idx"]

# save sample dataframe
sampleDF.to_csv(osp.join(ROOT, "raw", "sample_data.csv.gz"))

sampleDF

C:\Users\eek31\AppData\Local\Continuum\anaconda3\envs\my-rdkit-env\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\eek31\AppData\Local\Continuum\anaconda3\envs\my-rdkit-env\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,idx,smiles,homolumogap,old_idx
idx,,,,
0,0,SSc1ccccc1COC(=O)O,4.914376,1692148
1,1,NCCCc1nc2c(-c3ccsc3CC2)[nH]1,4.454504,1206752
2,2,CCCOc1cccc(c1)C(C(=O)OCC)O,5.504863,1289037
3,3,COC(=O)Cc1sc(cc1C#N)F,5.284451,439210
4,4,CC(Nc1ccc(cc1)C1CCCCC1)(C)C,5.273566,1674787
...,...,...,...,...
69995,69995,CCCCC1CC1S(=O)(=O)c1ccccc1,6.302157,3389179
69996,69996,Cc1ccc(o1)P(=O)(O)O,5.885823,3187295
69997,69997,CCCC(C(=O)O[Si](C)(C)C)C,7.428708,3106703


In [12]:
"""
Remap split_dict indices
"""
new_split_dict["train"] = np.array([idMapping[x] for x in new_split_dict["train"]])
new_split_dict["test"] = np.array([idMapping[x] for x in new_split_dict["test"]])
new_split_dict["valid"] = np.array([idMapping[x] for x in new_split_dict["valid"]])

torch.save(new_split_dict, osp.join(ROOT, "sample_split_dict.pt"))

## load data

In [72]:
def collate_dgl(samples):
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    labels = torch.stack(labels)

    return batched_graph, labels

In [24]:
from ogb.lsc import DglPCQM4MDataset, PCQM4MEvaluator

class SampleDglPCQM4MDataset(DglPCQM4MDataset):
    
    @property
    def raw_file_names(self):
        return 'sample_data.csv.gz'

    def prepare_graph(self):
        processed_dir = osp.join(self.folder, 'processed')
        raw_dir = osp.join(self.folder, 'raw')
        pre_processed_file_path = osp.join(processed_dir, 'dgl_data_processed')

        if osp.exists(pre_processed_file_path):        
            # if pre-processed file already exists
            self.graphs, label_dict = load_graphs(pre_processed_file_path)
            self.labels = label_dict['labels']
        else:
            # if pre-processed file does not exist
            
            if not osp.exists(osp.join(raw_dir, 'sample_data.csv.gz')):
                # if the raw file does not exist, then download it.
                self.download()

            data_df = pd.read_csv(osp.join(raw_dir, 'sample_data.csv.gz'))
            smiles_list = data_df['smiles']
            homolumogap_list = data_df['homolumogap']

            print('Converting SMILES strings into graphs...')
            self.graphs = []
            self.labels = []
            for i in tqdm(range(len(smiles_list))):

                smiles = smiles_list[i]
                homolumogap = homolumogap_list[i]
                graph = self.smiles2graph(smiles)
                
                assert(len(graph['edge_feat']) == graph['edge_index'].shape[1])
                assert(len(graph['node_feat']) == graph['num_nodes'])

                dgl_graph = dgl.graph((graph['edge_index'][0], graph['edge_index'][1]), num_nodes = graph['num_nodes'])
                dgl_graph.edata['feat'] = torch.from_numpy(graph['edge_feat']).to(torch.int64)
                dgl_graph.ndata['feat'] = torch.from_numpy(graph['node_feat']).to(torch.int64)

                self.graphs.append(dgl_graph)
                self.labels.append(homolumogap)

            self.labels = torch.tensor(self.labels, dtype=torch.float32)

            # double-check prediction target
            split_dict = self.get_idx_split()
            assert(all([not torch.isnan(self.labels[i]) for i in split_dict['train']]))
            assert(all([not torch.isnan(self.labels[i]) for i in split_dict['valid']]))
            assert(all([torch.isnan(self.labels[i]) for i in split_dict['test']]))

            print('Saving...')
            save_graphs(pre_processed_file_path, self.graphs, labels={'labels': self.labels})
        
    
    # just modify the get_idx_split function to call our new filename
    def get_idx_split(self):
        # NOTE: CHANGED split_dict.pt to sample_split_dict.pt
        split_dict = replace_numpy_with_torchtensor(torch.load(osp.join(self.folder, 'sample_split_dict.pt')))
        return split_dict

In [25]:
isSample = True #False #

if isSample:
    dataset = SampleDglPCQM4MDataset(root = 'dataset/')
else:
#     !rm dataset/pcqm4m_kddcup2021/processed/geometric_data_processed.pt
    dataset = DglPCQM4MDataset(root = 'dataset/')

  0%|▎                                                                           | 261/70000 [00:00<00:54, 1291.29it/s]

Converting SMILES strings into graphs...


100%|██████████████████████████████████████████████████████████████████████████| 70000/70000 [00:59<00:00, 1181.43it/s]


Saving...


In [38]:
split_idx = dataset.get_idx_split()
split_idx["train"] = split_idx["train"].type(torch.LongTensor)
split_idx["test"] = split_idx["test"].type(torch.LongTensor)
split_idx["valid"] = split_idx["valid"].type(torch.LongTensor)

split_idx["train"], split_idx["test"], split_idx["valid"]

(tensor([    0,     1,     2,  ..., 49997, 49998, 49999]),
 tensor([50000, 50001, 50002,  ..., 59997, 59998, 59999]),
 tensor([60000, 60001, 60002,  ..., 69997, 69998, 69999]))

In [39]:
evaluator = PCQM4MEvaluator()

In [86]:
# get args from main_gnn CLI
class Argument(object):
    name = "args"
    
args = Argument()
args.batch_size = 256
args.num_workers = 0
args.num_layers = 5
args.emb_dim = 600
args.drop_ratio = 0
args.graph_pooling = "sum"
args.checkpoint_dir = "checkpoint"
args.device = 0
args.log_dir = "models/gin-virtual/log"
args.checkpoint_dir = "models/gin-virtual/checkpoint"
args.train_subset = False
args.epochs = 10
args.save_test_dir = "models/gin-virtual/test"

In [58]:
device = torch.device("cuda:" + str(args.device)) if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda', index=0)

In [73]:
train_loader = DataLoader(dataset[split_idx["train"]], batch_size=args.batch_size, shuffle=True, num_workers = args.num_workers, collate_fn=collate_dgl)
valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=args.batch_size, shuffle=False, num_workers = args.num_workers, collate_fn=collate_dgl)
test_loader = DataLoader(dataset[split_idx["test"]], batch_size=args.batch_size, shuffle=False, num_workers = args.num_workers, collate_fn=collate_dgl)

In [74]:
shared_params = {
    'num_layers': args.num_layers,
    'emb_dim': args.emb_dim,
    'drop_ratio': args.drop_ratio,
    'graph_pooling': args.graph_pooling
}


## default model

In [75]:
from gnn import GNN

In [76]:
model = GNN(gnn_type='gin', virtual_node=True, **shared_params).to(device)

In [77]:
# check if checkpoint exist -> load model
checkpointFile = os.path.join(args.checkpoint_dir, 'checkpoint.pt')
if os.path.exists(checkpointFile):
    # load weights
    print("Loading existing weights from {}".format(checkpointFile))
    checkpointData = torch.load(checkpointFile)
    model.load_state_dict(checkpointData["model_state_dict"])

In [89]:
from main import train, eval, test

In [91]:
num_params = sum(p.numel() for p in model.parameters())
print(f'#Params: {num_params}')

optimizer = optim.Adam(model.parameters(), lr=0.001)

if args.log_dir is not '':
    writer = SummaryWriter(log_dir=args.log_dir)

best_valid_mae = 1000

if args.train_subset:
    scheduler = StepLR(optimizer, step_size=300, gamma=0.25)
    args.epochs = 1000
else:
    scheduler = StepLR(optimizer, step_size=30, gamma=0.25)

for epoch in range(1, args.epochs + 1):
    print("=====Epoch {}".format(epoch))
    print('Training...')
    train_mae = train(model, device, train_loader, optimizer)

    print('Evaluating...')
    valid_mae = eval(model, device, valid_loader, evaluator)

    print({'Train': train_mae, 'Validation': valid_mae})

    if args.log_dir is not '':
        writer.add_scalar('valid/mae', valid_mae, epoch)
        writer.add_scalar('train/mae', train_mae, epoch)

    if valid_mae < best_valid_mae:
        best_valid_mae = valid_mae
        if args.checkpoint_dir is not '':
            print('Saving checkpoint...')
            checkpoint = {'epoch': epoch, 'model_state_dict': model.state_dict(),
                          'optimizer_state_dict': optimizer.state_dict(),
                          'scheduler_state_dict': scheduler.state_dict(), 'best_val_mae': best_valid_mae,
                          'num_params': num_params}
            torch.save(checkpoint, os.path.join(args.checkpoint_dir, 'checkpoint.pt'))

        if args.save_test_dir is not '':
            print('Predicting on test data...')
            y_pred = test(model, device, test_loader)
#             print('Saving test submission file...')
#             evaluator.save_test_submission({'y_pred': y_pred}, args.save_test_dir)

    scheduler.step()

    print(f'Best validation MAE so far: {best_valid_mae}')

Iteration:   0%|                                                                               | 0/196 [00:00<?, ?it/s]

#Params: 6656406
=====Epoch 1
Training...


Iteration:   5%|███▌                                                                    | 2/40 [00:00<00:02, 17.39it/s]

Evaluating...


Iteration:   0%|                                                                                | 0/40 [00:00<?, ?it/s]

...

{'Train': 0.45975067101571027, 'Validation': 0.5335384607315063}
Saving checkpoint...
Predicting on test data...


Iteration:   1%|▎                                                                      | 1/196 [00:00<00:33,  5.82it/s]

Best validation MAE so far: 0.5335384607315063
=====Epoch 3
Training...


Iteration:  43%|██████████████████████████████▎                                       | 85/196 [00:13<00:18,  6.15it/s]


KeyboardInterrupt: 